<a href="https://colab.research.google.com/github/Rishit-dagli/SurpriseHacks/blob/main/notebooks/plant_disseases_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sun Apr  4 11:30:45 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Identify Plant Diseases

We use the PlantVillage dataset [1] by Hughes et al. consists of about 87,000 healthy and unhealthy leaf images divided into 38 categories by species and disease. Here we provide a subset of our experiments on working with this data. We also end up transfer learning from MobileNet [2] and use the weights from pre-training on ImageNet.

In this notebook we download the TensorFlow SavedModel we created in the prequel notebook and try to perform inference on a single image with the model.

![](https://github.com/Rishit-dagli/SurpriseHacks/blob/main/images/dataset-examples.png?raw=true)

Figure 1. PlantVillage Dataset Samples

## References

[1] Hughes, David P., and Marcel Salathe. “An Open Access Repository of Images on Plant Health to Enable the Development of Mobile Disease Diagnostics.” ArXiv:1511.08060 [Cs], Apr. 2016. arXiv.org, http://arxiv.org/abs/1511.08060.

[2] Howard, Andrew G., et al. “MobileNets: Efficient Convolutional Neural Networks for Mobile Vision Applications.” ArXiv:1704.04861 [Cs], Apr. 2017. arXiv.org, http://arxiv.org/abs/1704.04861.

## Setup

In [2]:
import tensorflow as tf
from PIL import Image
import json
import numpy as np
from google.colab import files

## Download the model

In [3]:
!git clone https://github.com/Rishit-dagli/SurpriseHacks

Cloning into 'SurpriseHacks'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 125 (delta 59), reused 81 (delta 36), pack-reused 0
Receiving objects: 100% (125/125), 13.49 MiB | 39.48 MiB/s, done.
Resolving deltas: 100% (59/59), done.


In [4]:
%cd SurpriseHacks/

/content/SurpriseHacks


In [5]:
class_indices_path = "./models/class_indices.json"
model_path = "./models/plant_disease.h5"

model = tf.keras.models.load_model(model_path)
with open(class_indices_path) as json_file:
  class_indices = json.load(json_file)

We would also need to format the class indices

In [6]:
class_indices = {v: k for (k, v) in class_indices.items()}

## Running the Model

Let's now take a look at actually running a prediction using the model. This code will allow you to choose 1 or more files from your file system, it will then upload them, and run them through the model, giving an indication of whether the object is a horse or a human.

In [7]:
uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/SurpriseHacks/' + fn

  image = Image.open(path)
  image = image.resize((224, 224))
  image = np.expand_dims(np.asarray(image), axis=0) / 255

  predictions = model(image)

  predicted_label = str(class_indices[np.argmax(predictions)])
  species, disease = predicted_label.split("__")

  print(f"{fn} is of a {species} plant")
  if disease == "healthy":
    print(f"{fn} is healthy ✔")
  else:
    disease_name = disease.replace("_", " ")
    print(f"{fn} is suffering from {disease_name}")

Saving AppleCedarRust1.png to AppleCedarRust1.png
Saving AppleScab1.png to AppleScab1.png
AppleCedarRust1.png is of a Apple plant
AppleCedarRust1.png is suffering from  Cedar apple rust
AppleScab1.png is of a Apple plant
AppleScab1.png is suffering from  Apple scab
